In [1]:
import torch
import torchvision as tv
import os
from transformer_flow import Model
import utils
import pathlib

os.environ["CUDA_VISIBLE_DEVICES"] = "4" # set GPU

utils.set_random_seed(100)
notebook_output_path = pathlib.Path('runs/notebook')

In [2]:
dataset = 'mnist'
num_classes = 10
img_size = 28
channel_size = 1

# we use a small model for fast demonstration, increase the model size for better results
patch_size = 4
channels = 128
blocks = 4
layers_per_block = 4
# try different noise levels to see its effect
noise_std = 0.1

batch_size = 256
lr = 5e-4
# increase epochs for better results
epochs = 100
sample_freq = 5

if torch.cuda.is_available():
    device = 'cuda' 
elif torch.backends.mps.is_available():
    device = 'mps' # if on mac
else:
    device = 'cpu' # if mps not available
print(f'using device {device}')

fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
# fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channels, device=device)
fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

transform = tv.transforms.Compose([
    tv.transforms.Resize((img_size, img_size)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5,), (0.5,))
])
data = tv.datasets.MNIST('.', transform=transform, train=True, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
              num_classes=num_classes).to(device)

optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, len(data_loader), epochs * len(data_loader), 1e-6, lr)

model_name = f'{patch_size}_{channels}_{blocks}_{layers_per_block}_{noise_std:.2f}_sqa_xibo_diaoda'
sample_dir = notebook_output_path / f'{dataset}_samples_{model_name}'
ckpt_file = notebook_output_path / f'{dataset}_model_{model_name}.pth'
sample_dir.mkdir(exist_ok=True, parents=True)

all_S = []

for epoch in range(epochs):
    losses = 0
    for x, y in data_loader:
        x = x.to(device)
        eps = noise_std * torch.randn_like(x)
        x = x + eps
        y = y.to(device)
        optimizer.zero_grad()
        z, outputs, logdets = model(x, y)
        loss = model.get_loss(z, logdets)
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        losses += loss.item()

        # print the singular values of the matrix of the x embedder
        # try:
        #   W = model.x_embedder.weight
        #   S = torch.linalg.svdvals(W, driver='gesvd')
        #   all_S.append(S)
        # except:
        #     print(f"there is error in calculating the singular values")
        #     print(f"matrix: {W.shape}")
        #     print(f"matrix: {W}")
        #     print(f"loss: {loss}")
        #     loss=torch.tensor(float('nan'), device=device)
        # if torch.isnan(loss): 
            # for i in range(1,6):
            #     print(f'singular value of S[-{i}]: {all_S[-i]}')
            # assert False
          
        # print(f'singular values {S.tolist()}')

    print(f"epoch {epoch} lr {optimizer.param_groups[0]['lr']:.6f} loss {losses / len(data_loader):.4f}")
    print('layer norms', ' '.join([f'{z.pow(2).mean():.4f}' for z in outputs]))
    print(f'logdet: {logdets.mean():.4f}, prior p: {0.5 * z.pow(2).mean():.4f}')
    if (epoch + 1) % sample_freq == 0:
        with torch.no_grad():
            samples = model.reverse(fixed_noise, fixed_y)
        tv.utils.save_image(samples, sample_dir / f'samples_{epoch:03d}.png', normalize=True, nrow=10)
        tv.utils.save_image(model.unpatchify(z[:100]), sample_dir / f'latent_{epoch:03d}.png', normalize=True, nrow=10)
        print('sampling complete')
torch.save(model.state_dict(), ckpt_file)

using device cuda
Number of parameters: 3.23M
epoch 0 lr 0.000500 loss -1.2519
layer norms 0.7683 1.1461 0.7098 0.7955
logdet: 2.1460, prior p: 0.3977
epoch 1 lr 0.000500 loss -1.9137
layer norms 0.7073 0.7623 0.6817 1.0357
logdet: 2.5509, prior p: 0.5178
epoch 2 lr 0.000499 loss -2.1119
layer norms 0.6204 0.6759 0.6670 0.9696
logdet: 2.6671, prior p: 0.4848
epoch 3 lr 0.000499 loss -2.1971
layer norms 0.5766 0.6280 0.6545 0.9932
logdet: 2.7352, prior p: 0.4966
epoch 4 lr 0.000498 loss -2.2347
layer norms 0.6274 0.6766 0.6919 0.9667
logdet: 2.6963, prior p: 0.4833
sampling complete
epoch 5 lr 0.000497 loss -2.2686
layer norms 0.5710 0.6360 0.6957 0.9699
logdet: 2.7886, prior p: 0.4850
epoch 6 lr 0.000495 loss -2.2970
layer norms 0.5645 0.6411 0.7010 0.9676
logdet: 2.8060, prior p: 0.4838
epoch 7 lr 0.000494 loss -2.3163
layer norms 0.5597 0.6447 0.6990 1.0219
logdet: 2.8386, prior p: 0.5110
epoch 8 lr 0.000492 loss -2.3323
layer norms 0.5532 0.6364 0.6808 1.0166
logdet: 2.8344, prior p

In [3]:
# now we can also evaluate the model by turning it into a classifier with Bayes rule, p(y|x) = p(y)p(x|y)/p(x)
data = tv.datasets.MNIST('.', transform=transform, train=False, download=False)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False)
num_correct = 0
num_examples = 0
for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    eps = noise_std * torch.randn_like(x)
    x = x.repeat(num_classes, 1, 1, 1)
    y_ = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, y.size(0)).flatten()
    with torch.no_grad():
        z, outputs, logdets = model(x, y_)
        losses = 0.5 * z.pow(2).mean(dim=[1, 2]) - logdets # keep the batch dimension
        pred = losses.reshape(num_classes, y.size(0)).argmin(dim=0)
    num_correct += (pred == y).sum()
    num_examples += y.size(0)
print(f'Accuracy %{100 * num_correct / num_examples:.2f}')

Accuracy %97.11
